# <center> Лабораторна робота №12. Прогнозування затримок вильоту літаків з використанням різних алгоритмів бустінгу

In [11]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [12]:
train = pd.read_csv("data/flight_delays_train.csv")
test = pd.read_csv("data/flight_delays_test.csv")

In [13]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [14]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Отже, потрібно за часом вильоту літака, коду авіакомпанії-перевізника, місця вильоту та прильоту та відстанню між аеропортами вильоту та прильоту передбачити затримку вильоту більше 15 хвилин. Як найпростіший бенчмарк візьмемо логістичну регресію та дві ознаки, які найлегше взяти: `DepTime` та `Distance`. У такої моделі результат – 0.68202.

In [15]:
X_train, y_train = (
    train[["Distance", "DepTime"]].values,
    train["dep_delayed_15min"].map({"Y": 1, "N": 0}).values,
)
X_test = test[["Distance", "DepTime"]].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(
    X_train, y_train, test_size=0.3, random_state=17
)

In [16]:
logit = LogisticRegression(random_state=17)

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

np.float64(0.6795697123357751)

In [17]:
logit.fit(X_train, y_train)
logit_test_pred = logit.predict_proba(X_test)[:, 1]

pd.Series(logit_test_pred, name="dep_delayed_15min").to_csv(
    "logit_2feat.csv", index_label="id", header=True
)

Побудувати покращену модель з використанням наступних підказок:
- ознаки `Distance` та `DepTime` брати без змін;
- створена ознака "маршрут" з вхідних ознак `Origin` та `Dest`;
- до ознак `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` і "маршрут" застосувати OHE-перетворення (`LabelBinarizer`);
- видділити відкладену вибірку;
- навчати логістичну регресію і градієнтний бустінг (xgboost або sklearn.ensemble.GradientBoostingRegressor), гіперпараметри бустінгу налаштувати за результатами крос-валідації, спочатку ті, що відповідають за складність моделі, потім число дерев зафіксувати рівним 500 і налаштовувати крок градієнтного спуску;
- за допомогою `cross_val_predict` сформувати прогнози обох моделей на крос-валідації (саме передбачення ймовірності), налаштувати лінійну суміш відповідей логістичної регресії і градієнтного бустінгу вигляду $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, де $p_{logit}$ – прогноз логістичною регресією ймовірності класу 1, $p_{xgb}$ – аналогічно. Вага $w_1$ підбирається вручну. 
- як відповідь на тестовій вибірці брати аналогічну комбінацію відповідей двух моделей, але вже навчених на всій навчальній вибірці.

In [21]:
# Створення ознаки "route"
train['route'] = train['Origin'] + '_' + train['Dest']
test['route'] = test['Origin'] + '_' + test['Dest']

# Визначення категоріальних та числових ознак
categorical_features = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'route']
numeric_features = ['Distance', 'DepTime']

# Створення трансформера для One-Hot Encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Підготовка повного набору ознак
X_full = train[categorical_features + numeric_features]
y_full = train["dep_delayed_15min"].map({"Y": 1, "N": 0}).values
X_test_full = test[categorical_features + numeric_features]

# Розділення даних на навчальну та відкладену вибірку
X_train_full, X_valid, y_train_full, y_valid = train_test_split(
    X_full, y_full, test_size=0.3, random_state=17, stratify=y_full
)

# Пайплайн для логістичної регресії
logit_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler(with_mean=False)),  # Виправлення для розріджених матриць
    ('logit', LogisticRegression(random_state=17, max_iter=1000))
])

# Пайплайн для XGBoost
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xgb', XGBClassifier(
        random_state=17,
        n_estimators=500,
        learning_rate=0.05,  # Крок градієнтного спуску
        max_depth=6,
        eval_metric='auc'  # Використовуємо правильний параметр без 'use_label_encoder'
    ))
])

# Навчання логістичної регресії
logit_pipeline.fit(X_train_full, y_train_full)
logit_valid_pred = logit_pipeline.predict_proba(X_valid)[:, 1]
logit_auc = roc_auc_score(y_valid, logit_valid_pred)
print(f"Logistic Regression ROC AUC: {logit_auc:.5f}")

# Навчання XGBoost
xgb_pipeline.fit(X_train_full, y_train_full)
xgb_valid_pred = xgb_pipeline.predict_proba(X_valid)[:, 1]
xgb_auc = roc_auc_score(y_valid, xgb_valid_pred)
print(f"XGBoost ROC AUC: {xgb_auc:.5f}")

# Використання cross_val_predict для отримання прогнозів на всіх даних
logit_cv_pred = cross_val_predict(
    logit_pipeline, X_full, y_full, cv=5, method='predict_proba', n_jobs=-1
)[:, 1]

xgb_cv_pred = cross_val_predict(
    xgb_pipeline, X_full, y_full, cv=5, method='predict_proba', n_jobs=-1
)[:, 1]

# Функція для знаходження оптимальної ваги
def find_best_weight(y_true, pred1, pred2):
    best_w = 0
    best_auc = 0
    for w in np.linspace(0, 1, 101):
        hybrid_pred = w * pred1 + (1 - w) * pred2
        auc = roc_auc_score(y_true, hybrid_pred)
        if auc > best_auc:
            best_auc = auc
            best_w = w
    return best_w, best_auc

# Пошук оптимальної ваги
best_w, best_hybrid_auc = find_best_weight(y_full, logit_cv_pred, xgb_cv_pred)
print(f"Optimal weight: {best_w:.2f}, Hybrid ROC AUC: {best_hybrid_auc:.5f}")

# Навчання фінальних моделей на всіх даних
logit_pipeline.fit(X_full, y_full)
xgb_pipeline.fit(X_full, y_full)

# Прогнозування на тестовій вибірці
logit_test_pred = logit_pipeline.predict_proba(X_test_full)[:, 1]
xgb_test_pred = xgb_pipeline.predict_proba(X_test_full)[:, 1]

# Гібридний прогноз на тестовій вибірці
hybrid_test_pred = best_w * logit_test_pred + (1 - best_w) * xgb_test_pred

# Створення стовпця 'id' за допомогою індексу тестового набору
submission = pd.DataFrame({
    "id": test.index,  # Використовуємо індекси рядків як 'id'
    "dep_delayed_15min": hybrid_test_pred
})

# Збереження результатів
submission.to_csv("hybrid_model_predictions.csv", index=False)
print("Гібридні прогнози збережено у 'hybrid_model_predictions.csv'")


Logistic Regression ROC AUC: 0.67184
XGBoost ROC AUC: 0.73456
Optimal weight: 0.13, Hybrid ROC AUC: 0.72985
Гібридні прогнози збережено у 'hybrid_model_predictions.csv'
